# Computação Eficiente

Python é linguagem OO pensada como interface para sistemas de computação complexos. Assim, operações aparentemente simples escondem algoritmos sofisticados:

In [ ]:
2+1

In [ ]:
type(2)

In [ ]:
(2).__add__(1)

_Observação_: use dir() para descobrir os atributos e métodos disponíveis para um objeto.

In [ ]:
', '.join(dir(2))

In [ ]:
import numpy as np

In [ ]:
v1 = np.random.rand(100)
v2 = np.random.rand(100)
v3 = v1 + v2
print(v3)

In [ ]:
m1 = np.random.rand(100, 15)
m2 = np.random.rand(15, 110)

In [ ]:
def matmul(m1, m2):
    r = np.zeros((m1.shape[0], m2.shape[1]))
    for i in range(m1.shape[0]):
        for j in range(m2.shape[1]):
            for k in range(m2.shape[0]):
                r[i][j] += m1[i][k] * m2[k][j]
    return r

In [ ]:
%timeit matmul(m1, m2)

In [ ]:
%timeit np.matmul(m1, m2)

In [ ]:
177000/21.4

Será que esta diferença de desempenho se deve ao mau uso do Python no primeiro exemplo? Esse ganho de eficiência talvez não exista ao se comparar com o código tivesse sido escrito em C e compilado de forma otimizada. Vamos ver?

Comparação de multiplicação de matrizes em Numpy, Python (C-Python) e C

<img src="images/TLhDi" alt="NumpyVersusC" style="width: 500px;"/>

Por que, na comparação acima, o Numpy foi tão rápido? Mais rápido mesmo que um código puro escrito em C? Basicamente, porque o código em numpy corresponde a versões otimizadas do melhor algotimo (prático) conhecido para o problema, escrito originalmente em Fortran para o BLAS. Abaixo, podemos ver a implementação do algoritmo de Strassen -- $O(n^{2.83})$, usado pelo numpy. Note o detalhe do _loop unrolling_ para acelerar o processamento em um hardware real.

```C

/*  -- translated by f2c (version 19940927).
   You must link the resulting object file with the libraries:
	-lf2c -lm   (in that order)
*/

#include "f2c.h"

doublereal sdot_(integer *n, real *sx, integer *incx, real *sy, integer *incy)
{


    /* System generated locals */
    integer i__1;
    real ret_val;

    /* Local variables */
    static integer i, m;
    static real stemp;
    static integer ix, iy, mp1;


/*     forms the dot product of two vectors.   
       uses unrolled loops for increments equal to one.   
       jack dongarra, linpack, 3/11/78.   
       modified 12/3/93, array(1) declarations changed to array(*)   


    
   Parameter adjustments   
       Function Body */
#define SY(I) sy[(I)-1]
#define SX(I) sx[(I)-1]


    stemp = 0.f;
    ret_val = 0.f;
    if (*n <= 0) {
	return ret_val;
    }
    if (*incx == 1 && *incy == 1) {
	goto L20;
    }

/*        code for unequal increments or equal increments   
            not equal to 1 */

    ix = 1;
    iy = 1;
    if (*incx < 0) {
	ix = (-(*n) + 1) * *incx + 1;
    }
    if (*incy < 0) {
	iy = (-(*n) + 1) * *incy + 1;
    }
    i__1 = *n;
    for (i = 1; i <= *n; ++i) {
	stemp += SX(ix) * SY(iy);
	ix += *incx;
	iy += *incy;
/* L10: */
    }
    ret_val = stemp;
    return ret_val;

/*        code for both increments equal to 1   


          clean-up loop */

L20:
    m = *n % 5;
    if (m == 0) {
	goto L40;
    }
    i__1 = m;
    for (i = 1; i <= m; ++i) {
	stemp += SX(i) * SY(i);
/* L30: */
    }
    if (*n < 5) {
	goto L60;
    }
L40:
    mp1 = m + 1;
    i__1 = *n;
    for (i = mp1; i <= *n; i += 5) {
	stemp = stemp + SX(i) * SY(i) + SX(i + 1) * SY(i + 1) + SX(i + 2) * 
		SY(i + 2) + SX(i + 3) * SY(i + 3) + SX(i + 4) * SY(i + 4);
/* L50: */
    }
L60:
    ret_val = stemp;
    return ret_val;
} /* sdot_ */

```

### O Tensorflow é o numpy para máquinas vetoriais

Contudo, usa uma abordagem estática. Ou seja, o grafo de computação é declarado primeiro e, então, compilado e executado.

In [ ]:
import tensorflow as tf

In [ ]:
t1 = tf.constant(np.random.rand(100, 15), dtype = tf.float32)
t2 = tf.constant(np.random.rand(15, 110), dtype = tf.float32)

In [ ]:
t3 = tf.matmul(t1, t2)

In [ ]:
t3

In [ ]:
with tf.Session() as s:
    print(s.run(t3))

In [ ]:
def sigmoid(t):
    return 1.0 / (1.0 + tf.exp(-t))

In [ ]:
t1 = tf.constant(np.random.rand(100, 15), dtype = tf.float32)
t2 = tf.constant(np.random.rand(15, 110), dtype = tf.float32)

In [ ]:
t1s = sigmoid(t1)

In [ ]:
t1s

In [ ]:
with tf.Session() as s:
    print(s.run(t1s))

In [ ]:
t1 = tf.placeholder(shape = (100, 15), dtype = tf.float32)
t2 = tf.placeholder(shape = (15, 110), dtype = tf.float32)

In [ ]:
t3 = tf.matmul(t1, t2)

In [ ]:
t3

In [ ]:
with tf.Session() as s:
    print(s.run(t3, feed_dict = {t1: m1, t2: m2}))
    m1 = 2 * m1
    m2 = 3 * m2
    print(s.run(t3, feed_dict = {t1: m1, t2: m2}))

O problema com essa abordagem é que ela ainda é relativamente baixo nível. Logo, não é produtiva em termos de prototipação. Assim, surge a necessidade de APIs de mais alto nível ainda, como o Keras.